# Project 3

The goal of this project is to Compare differnt Rnn models to determine which is the fastest or most accurate at basic stock prediction. This will be accomplished by using Amazon stock history to create a model that will determine if a stock has gone up in price or down in price. It is thus a binary classifier. 

Once the fastest model has been chosen then I will attmept to do a price prediction with that model.

In [12]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Sequential

from tensorflow.strings import regex_replace
from tensorflow.keras.preprocessing import text_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as pdta
import tqdm
import json
import os
import shutil
import zipfile
import os
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
numGPUs = len(tf.config.experimental.list_physical_devices('GPU'))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.python.client import device_lib
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
print('Num GPUs Available: ', numGPUs)

if numGPUs > 0:
    print(tf.test.gpu_device_name())
    print(device_lib.list_local_devices()[1].physical_device_desc)


Num GPUs Available:  0


2022-12-10 22:39:43.045576: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/nate/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [13]:
df = yf.download(tickers='AMZN',start = '1990-01-01', end= '2022-12-07')
display(df)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,0.121875,0.125000,0.096354,0.097917,0.097917,1443120000
1997-05-16,0.098438,0.098958,0.085417,0.086458,0.086458,294000000
1997-05-19,0.088021,0.088542,0.081250,0.085417,0.085417,122136000
1997-05-20,0.086458,0.087500,0.081771,0.081771,0.081771,109344000
1997-05-21,0.081771,0.082292,0.068750,0.071354,0.071354,377064000
...,...,...,...,...,...,...
2022-11-30,92.470001,96.540001,91.529999,96.540001,96.540001,102628200
2022-12-01,96.989998,97.230003,94.919998,95.500000,95.500000,68488000
2022-12-02,94.480003,95.360001,93.779999,94.129997,94.129997,72427000


Alright so after loading the history of Amazons stocks in to the dataframe we have a bunch of columns. At first glance the data column is not gonna be needed. the other columns on the other hand all have useful data to them. However the Volumn column is going to be dropped. While it is useful to know how many stocks were sold in a day, these numbers appear to be wrong. Even if they are not wrong they are so large that if included them in the model everything else would look like zero so it would make the dataset almost useless. therefore the volumn and the data column will be dropped. The adjusted close will also be droped as it looks like a duplicate of the close price. I will also be adding some technical indicators that are typicall helpful when doing these kind of predictions. These indicator are the relative strength index which will be a measure of how much stocks gain or lose the standard period is around 14 days, the next few columns will be some Exponential moving averages of different lengths to represent different trends in stocks. these extra data points should be invaluable to the models over all accuracy. The averages typically have 10,50 and 200 day averages so that is what I will be doing here. These momentum indicators will be based on the close prices

In [14]:
#adding new columns and doing the math with pandas_ta
df["RSI"] = pdta.rsi(df['Close'], length=14)
df['EMA_s'] = pdta.ema(df['Close'], length=10)
df['EMA_m'] = pdta.ema(df['Close'], length=50)
df['EMA_l'] = pdta.ema(df['Close'], length=200)

#now we need to make out label so to do this we need to know if a stocks
#price has gone up or down so we will be making a change column using the close data
#we then need to shift this down one so that the next day will have the change this is done by the shift(-1)

df['Change'] = df['Adj Close']-df['Open']
df['Change'] = df["Change"].shift(-1)

#now using the change we can make our label

df["Up/Down"] = [1 if df['Change'][i]>0 else 0 for i in range(len(df))]


df['NextDayAdjClose'] = df["Adj Close"].shift(-1)
df = df.dropna()
df = df.drop(columns=['Volume' ,'Close'])
display(df)


,Open,High,Low,Adj Close,RSI,EMA_s,EMA_m,EMA_l,Change,Up/Down,NextDayAdjClose
Date,,,,,,,,,,,
1998-03-02,0.316146,0.321875,0.312500,0.317708,76.957593,0.287125,0.253435,0.175827,-0.015625,0,0.298958
1998-03-03,0.314583,0.315625,0.297917,0.298958,64.204877,0.289276,0.255220,0.177052,0.005990,1,0.305990
1998-03-04,0.300000,0.309375,0.291667,0.305990,66.450323,0.292315,0.257211,0.178335,0.013542,1,0.312500
1998-03-05,0.298958,0.325781,0.296875,0.312500,68.425050,0.295985,0.259379,0.179670,0.004688,1,0.319792
1998-03-06,0.315104,0.320833,0.307292,0.319792,70.518307,0.300314,0.261748,0.181064,0.027604,1,0.347917
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,94.040001,94.410004,91.440002,92.419998,38.539036,94.160898,105.071174,124.659814,4.070000,1,96.540001
2022-11-30,92.470001,96.540001,91.529999,96.540001,46.409553,94.593463,104.736618,124.380015,-1.489998,0,95.500000
2022-12-01,96.989998,97.230003,94.919998,95.500000,44.848309,94.758288,104.374398,124.092652,-0.350006,0,94.129997


In [15]:
# copy the data
df_max_scaled = df.copy()
conversionMax = df['NextDayAdjClose'].abs().max()
print(conversionMax)
# apply normalization techniques
for column in df_max_scaled.columns:
    
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max()
      
# view normalized data
display(df_max_scaled)
print(0.001602*conversionMax)

186.57049560546875


,Open,High,Low,Adj Close,RSI,EMA_s,EMA_m,EMA_l,Change,Up/Down,NextDayAdjClose
Date,,,,,,,,,,,
1998-03-02,0.001689,0.001706,0.001691,0.001703,0.816837,0.001579,0.001449,0.001048,-0.001356,0.0,0.001602
1998-03-03,0.001680,0.001673,0.001612,0.001602,0.681478,0.001591,0.001460,0.001055,0.000520,1.0,0.001640
1998-03-04,0.001603,0.001640,0.001578,0.001640,0.705312,0.001608,0.001471,0.001063,0.001175,1.0,0.001675
1998-03-05,0.001597,0.001727,0.001606,0.001675,0.726272,0.001628,0.001483,0.001071,0.000407,1.0,0.001714
1998-03-06,0.001683,0.001701,0.001662,0.001714,0.748490,0.001652,0.001497,0.001079,0.002396,1.0,0.001865
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,0.502350,0.500440,0.494699,0.495362,0.409058,0.517820,0.600911,0.742765,0.353283,1.0,0.517445
2022-11-30,0.493964,0.511730,0.495186,0.517445,0.492596,0.520199,0.598997,0.741098,-0.129334,0.0,0.511871
2022-12-01,0.518109,0.515388,0.513527,0.511871,0.476025,0.521105,0.596926,0.739386,-0.030381,0.0,0.504528


0.29888593395996094


In [19]:
allX = df_max_scaled.drop(columns=['NextDayAdjClose', 'Change', 'Up/Down'])
allY = df_max_scaled['NextDayAdjClose']

# I can rewrite this maybe although i dont think I need to

In [26]:
numpy_scaled = df_max_scaled.to_numpy()

x_data =[]

numDays = 10 #this is the number of days feed in before a prediction

print(numpy_scaled.shape)
for x in range(8): #want to keep the first 8 columns for the data
    x_data.append([])
    for y in range(numDays, numpy_scaled.shape[0]):
        x_data[x].append(numpy_scaled[y-numDays:y,x])

x_data = np.moveaxis(x_data, [0],[2])

x_data, y = np.array(x_data), np.array(numpy_scaled[numDays:,-1])
y=np.reshape(y,(len(y),1))
print(x_data.shape,"\n",y.shape)





(6234, 11)
(6224, 10, 8) 
 (6224, 1)
[0.33281299]


In [29]:
print(6234*.6)
print(6234-3740)
print(3740*.5)


3740.3999999999996
2494
1870.0


In [31]:
trainX, valX, testX = x_data[:3740], x_data[3740:5610], x_data[5610:]
trainY, valY, testY = y[:3740], y[3740:5610], y[5610:]

print(trainX.shape, valX.shape, testX.shape)
print(trainY.shape, valY.shape, testY.shape)

(3740, 10, 8) (1870, 10, 8) (614, 10, 8)
(3740, 1) (1870, 1) (614, 1)
